In [26]:
using BenchmarkTools
using Base.Test

In [16]:
Base.Test.@inferred nowrap.satisfies(nowrap.ContainsLetter('h'), stat)

true

In [41]:
module wrap

import FunctionWrappers: FunctionWrapper

immutable Statistics
    word::String
end

immutable Feature
    f::FunctionWrapper{Bool, Tuple{String}}
end

(feat::Feature)(stat) = feat.f(stat)
satisfies(feat::Feature, stat) = feat.f(stat)

const FEATURES = Feature[]

function allfeatures()
    return FEATURES
end

for l in 'a':'z'
    push!(FEATURES, Feature(word -> l in word))
end

for l in 'a':'z'
    for j in 1:26
        push!(FEATURES, Feature(word -> length(word) >= j && word[j] == l))
    end
end

end

wrap

In [42]:
word = "hello"
# stat = wrap.Statistics(word)
fs = wrap.allfeatures()
@benchmark [f($word) for f in $fs]

BenchmarkTools.Trial: 
  memory estimate:  11.80 kb
  allocs estimate:  705
  --------------
  minimum time:     128.010 μs (0.00% GC)
  median time:      134.542 μs (0.00% GC)
  mean time:        142.001 μs (0.43% GC)
  maximum time:     1.920 ms (84.36% GC)
  --------------
  samples:          10000
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [43]:
f = fs[1]

wrap.Feature(FunctionWrappers.FunctionWrapper{Bool,Tuple{String}}(Ptr{Void} @0x000000031fb318f0,Ptr{Void} @0x000000011982a080,Base.RefValue{wrap.##1#2}(wrap.#1),wrap.##1#2))

In [45]:
@code_warntype f(word)

Variables:
  feat::wrap.Feature
  stat::String
  ptr::Ptr{Void}
  objptr::Ptr{Void}

Body:
  begin 
      SSAValue(0) = (Core.getfield)(feat::wrap.Feature,:f)::FunctionWrappers.FunctionWrapper{Bool,Tuple{String}}
      # meta: location /Users/rdeits/.julia/v0.5/FunctionWrappers/src/FunctionWrappers.jl FunctionWrapper 78
      # meta: location /Users/rdeits/.julia/v0.5/FunctionWrappers/src/FunctionWrappers.jl do_ccall 63
      NewvarNode(:(objptr::Ptr{Void}))
      # meta: location /Users/rdeits/.julia/v0.5/FunctionWrappers/src/FunctionWrappers.jl # line 64: # line 65:
      ptr::Ptr{Void} = (Core.getfield)(SSAValue(0),:ptr)::Ptr{Void} # line 66:
      unless ((Base.box)(UInt64,ptr::Ptr{Void}) === (Base.box)(UInt64,FunctionWrappers.C_NULL))::Bool goto 13 # line 68:
      ptr::Ptr{Void} = $(Expr(:invoke, LambdaInfo for reinit_wrapper(::FunctionWrappers.FunctionWrapper{Bool,Tuple{String}}), :(FunctionWrappers.reinit_wrapper), SSAValue(0)))
      13:  # line 70:
      ((Core.getfield)(Func

In [51]:
[f(word) for f in fs]

702-element Array{Bool,1}:
 false
 false
 false
 false
  true
 false
 false
  true
 false
 false
 false
  true
 false
     ⋮
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false

In [50]:
@code_warntype([f(word) for f in fs])

LoadError: LoadError: expression is not a function call, or is too complex for @code_warntype to analyze; break it down to simpler parts if possible
while loading In[50], in expression starting on line 1

In [33]:
@benchmark $f($stat)

BenchmarkTools.Trial: 
  memory estimate:  16.00 bytes
  allocs estimate:  1
  --------------
  minimum time:     65.524 ns (0.00% GC)
  median time:      68.619 ns (0.00% GC)
  mean time:        78.206 ns (0.77% GC)
  maximum time:     1.114 μs (91.29% GC)
  --------------
  samples:          10000
  evals/sample:     977
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [27]:
@inferred(fs[1](stat))

false